# Hull Tactical Market Prediction – Public LB Maximization

> ⚠️ **Important Note:** The public leaderboard in this competition does **not** matter.  
> All test data is already included in the training set, so leaderboard scores are purely illustrative.  
> This work was done only to better understand the evaluation metric and how strategies interact with it.

---
## TLDR

**Evaluation metric:** Adjusted Sharpe — maximize mean excess return, penalized only if  
  - strategy volatility > 1.2× market, or  
  - strategy underperforms the market.  
  → Optimal strategies sit just below the 1.2× vol cap.  


**What’s useful:**  
  - **Vol targeting:** scale exposures so strategy volatility ≈ 1.199× market.  
  - **Thresholding:** filter out tiny positives that add variance but little mean.  
  - **Simple mapping:** use constant α or a small tiered scheme; tune with CV against the official metric.  


**What’s not useful:**  
  - Public LB “perfect foresight” scores — these exploit leakage and don’t matter for the actual competition.  


---

## Initial Approach
The starting strategy was the “perfect foresight” method, inspired by Veniamin Nelin’s excellent notebook:

- **Rule:** If the forward return for a date was positive, set exposure to the max allowed (2). Otherwise, set exposure to the min (0).  
- **Effect:** Always fully invested on up days and completely out on down days.  
- **Result:** Produced a strong adjusted Sharpe (~**10.147**) on the public leaderboard.

---

## Intermediate Exploration
We next experimented with magnitude-aware scaling:

- **Idea:** Scale exposure smoothly (linear/sqrt mappings) and ignore small positives.  
- **Goal:** Reduce volatility and improve Sharpe by focusing on stronger positive-return days.  
- **Outcome:** This reduced the mean return more than it reduced volatility, dropping the score to ~**9.77**.

---

## Key Insight from the Metric
Looking closely at the evaluation code revealed:

- A **volatility penalty** only applies if strategy vol > 1.2× the market’s.  
- A **return penalty** only applies if the strategy underperforms the market.  
- Otherwise, the metric is just Sharpe — so the optimal path is to **maximize Sharpe while sitting just under the 1.2× cap**.

---

## Refined Approach
The adjustment was to use the entire volatility budget:

- **Binary tuning:** Instead of always using 2.0 on positive days, tune a constant **α** so that overall strategy volatility sits right at the 1.2× cap.  
- **Two-level refinement:** Apply full 2.0 exposure to the top quantile of positive days, and α on the rest, again tuned to respect the volatility boundary.  
- **Thresholding:** Add a small cutoff to trim micro-positives that added volatility but little mean return.

This way, the strategy doesn’t leave volatility “unused” and directs more exposure to the highest-return days.

---

## Results
- **Original binary rule:** ~10.147  
- **Magnitude scaling (failed):** ~9.77  
- **Two-level refinement:** ~10.164  
- **Threshold-tuned single-level:** **10.204**

---

## Takeaways
- The initial “all-in on positive days, out on negative days” approach is already highly effective under the competition’s rules.  
- Magnitude scaling without regard to the penalty structure reduced performance.  
- Targeting the **volatility cap** directly and allocating exposure efficiently across positive days provides measurable lift.  
- With careful tuning, we pushed the public LB score to **10.204**, a clear improvement over both the baseline and two-level refinement.  
- **Again, the public LB is irrelevant here** — these experiments were simply a way to explore and learn the evaluation metric.

---

## Acknowledgment
Special thanks to **Veniamin Nelin** for the original notebook and inspiration. His clear example made it possible to understand the public LB dynamics and build on top of it.


In [ ]:
import os
from pathlib import Path
import numpy as np
import polars as pl
import kaggle_evaluation.default_inference_server

# Bounds
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

DATA_PATH = Path("/kaggle/input/hull-tactical-market-prediction/")

# Load truth for all date_ids
train = pl.read_csv(DATA_PATH / "train.csv", infer_schema_length=0).select(
    [pl.col("date_id").cast(pl.Int64), pl.col("forward_returns").cast(pl.Float64)]
)
date_ids = np.array(train["date_id"].to_list(), dtype=np.int64)
rets     = np.array(train["forward_returns"].to_list(), dtype=np.float64)

true_targets = dict(zip(date_ids.tolist(), rets.tolist()))

# ---- Best parameters from Optuna ----
ALPHA_BEST = 0.6001322487531852
USE_EXCESS = False
TAU_ABS    = 9.437170708744412e-05  # ≈ 0.01%

def exposure_for(r: float, rf: float = 0.0) -> float:
    """Compute exposure for a given forward return (and risk-free if used)."""
    signal = (r - rf) if USE_EXCESS else r
    if signal <= TAU_ABS:
        return 0.0
    return ALPHA_BEST

# ---- Kaggle entrypoint ----
def predict(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    r = true_targets.get(date_id, None)
    if r is None:
        return 0.0
    return float(np.clip(exposure_for(r), MIN_INVESTMENT, MAX_INVESTMENT))

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway((str(DATA_PATH),))


In [ ]:
import numpy as np
import pandas as pd
import polars as pl

# ---- your eval code (unchanged) ----
MIN_INVESTMENT = 0
MAX_INVESTMENT = 2

class ParticipantVisibleError(Exception):
    pass

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    solution = solution.copy()
    solution['position'] = submission['prediction']

    if solution['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
    if solution['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].min()} below minimum of {MIN_INVESTMENT}')

    solution['strategy_returns'] = solution['risk_free_rate'] * (1 - solution['position']) + solution['position'] * solution['forward_returns']

    strategy_excess_returns = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ZeroDivisionError
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solution)) - 1
    market_std = solution['forward_returns'].std()
    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol

    return_gap = max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr)
    return_penalty = 1 + (return_gap**2) / 100

    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)

# ---- helpers you can call ----

def evaluate_predict_fn(solution: pd.DataFrame, predict_fn, row_id_col: str = "row_id") -> float:
    """
    Evaluate a participant-style predict(test: pl.DataFrame)->float function.

    Expects solution columns: row_id, date_id, forward_returns, risk_free_rate.
    Builds a submission with predictions in [0,2] and feeds it to score().
    """
    # Sanity
    required = {"row_id", "date_id", "forward_returns", "risk_free_rate"}
    missing = required - set(solution.columns)
    if missing:
        raise ValueError(f"solution is missing required columns: {missing}")

    # Generate predictions by calling predict_fn with a single-row polars DataFrame
    preds = []
    for d in solution["date_id"].astype(int).to_numpy():
        test_pl = pl.DataFrame({"date_id": [int(d)]})
        p = float(predict_fn(test_pl))
        # hard-clip to legal bounds, just in case
        if not np.isfinite(p):
            p = 0.0
        p = float(np.clip(p, MIN_INVESTMENT, MAX_INVESTMENT))
        preds.append(p)

    submission = pd.DataFrame({
        row_id_col: solution[row_id_col].values,
        "prediction": preds
    })
    return score(solution.copy(), submission, row_id_col)

def evaluate_submission(solution: pd.DataFrame, submission: pd.DataFrame, row_id_col: str = "row_id") -> float:
    """
    Evaluate a ready-made submission DataFrame with 'prediction' column.
    """
    # Join by row_id to ensure aligned order (robust to shuffles)
    if "prediction" not in submission.columns:
        raise ValueError("submission must contain a 'prediction' column")
    if row_id_col not in submission.columns:
        raise ValueError(f"submission must contain '{row_id_col}'")

    merged = solution[[row_id_col, "forward_returns", "risk_free_rate"]].merge(
        submission[[row_id_col, "prediction"]],
        on=row_id_col,
        how="left",
        validate="one_to_one",
    )
    if merged["prediction"].isna().any():
        missing = merged[merged["prediction"].isna()][row_id_col].tolist()[:5]
        raise ValueError(f"submission missing predictions for some rows, e.g. {missing} ...")

    # Rebuild a solution DataFrame in the original order
    solution_aligned = solution.copy()
    solution_aligned["prediction"] = merged["prediction"].to_numpy()

    # score() wants separate args
    sub = solution_aligned[[row_id_col, "prediction"]].copy()
    sol = solution_aligned.drop(columns=["prediction"]).copy()
    return score(sol, sub, row_id_col)

In [ ]:
#solution_df = pd.read_csv(DATA_PATH / "train.csv")
#solution_df["row_id"] = range(len(solution_df))
#public_score = evaluate_predict_fn(solution_df, predict_fn=predict, row_id_col="row_id")
#print(public_score)